# Review : DenseNet

>* edwith 강의 내용과 함께 SH Tsang의 [Review: DenseNet](https://towardsdatascience.com/review-densenet-image-classification-b6631a8ef803)을 번역한 글입니다.
>
>* [2107 ICCV, DenseNet](https://arxiv.org/pdf/1608.06993.pdf)

이번 글에서는 DenseNet(Dense Convolutional Network)에 대해 알아보자. DenseNet은 2017 CVPR에서 최고 논문 상을 받은 논문이며 Cornwell University, Tsinghua University, Facebook AI Research (FAIR)에서 만들었다. 

CNN이 깊어지면서 새로운 문제가 발생하기 시작했다. layer를 통과하면서 input에 대한 정보나 그래디언트가 사라지는 문제가 나타나는 점이며, 당시 논문들이 이 문제를 해결하는 방법을 찾으려고 노력했다. 이러한 논문에서 핵심포인트(key point)는 앞 layer과 뒤 layer 사이에 short path를 만드는 것이다.

DenseNet는 기존 ResNet과 달리 모든 layer를 연결하는 __Dense block__이라는 구조를 제안하고 있다.

## Index

1. [Dense Block](#a)
2. [DenseNet Architecture](#b)
3. [Advantages of DenseNet](#c)
4. [CIFAR & SVHN Small-Scale Dataset Results](#d)
5. [ImageNet Large-Scale Dataset Results](#e)
6. [Further Analysis on Feature Reuse](#f)

<a id='a'></a>
## 1. Dense Block

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*RfyAoe6Wlv4aLip2Y5Aw-Q.png" width="600">
    <br>
    <i>Standard ConvNet Concept</i>
</div>
<br>

기존의 ConvNet 구조에서는 다수의(multiple) convolution 층을 통과해서 high-level features를 얻을 수 있었다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*4wx7szWCBse9-7eemGQJSw.png" width="600">
    <br>
    <i>ResNet Concept</i>
</div>
<br>

__ResNet__에서는 그래디언트 전파(gradient propagation)를 향상시키기위해 앞 layer와 뒤 layer를 short path로 연결시키는 identity mapping 구조를 만들었다. Identity mapping은 단순하게 벡터를 더하는 방법인 요소별 연산(Element-wise addtion, [here](https://datascienceschool.net/view-notebook/3f44cfdda2874080a9aa6b034c71d5ec/))을 사용했으며, ResNet module의 output을 다른 module로 전달하는 알고리즘이라고 볼 수 있다. 

논문에서는 요소별 연산을 통해 다음 layer로 전달하는 과정이 정보의 흐름(information flow)를 방해하고 있다고 말하고 있다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*rmHdoPjGUjRek6ozH7altw.png" width="600">
    <br>
    <i>One Dense Block in DenseNet</i>
</div>
<br>

__DenseNet__에서는 이전 layers의 모든 출력값을 합쳐서 다음 layer의 입력값으로 사용한다. 여기서는 _Concatenation_이 사용된다. 즉 각각의 layer는 선행 layer로부터 결합된 정보(collective knowledge)를 받는다고 할 수 있다.<br>논문에서는 all layer를 연결하게되면 maximum information flow를 보장할 수 있다고 언급하고 있다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*P7tcHlzxm9Afg0Cejftv0g.png" width="600">
    <br>
    <i>Dense Block in DenseNet with Growth Rate k</i>
</div>
<br>

각 layer는 모든 선행 layer의 feature map값을 받기때문에, 네트워크를 가볍고 간결하게 구성할 수 있다. 즉 채널의 숫자를 줄일 수 있게된다. 그림에서 _growth rate k_는 각 layer에서 추가하는 채널의 숫자이다.

따라서 계산적, 메모리상으로 높은 효율성을 보여준다. 아래는 DenseNet의 forward propagation의 개념을 표현한 그림이다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*9ysRPSExk0KvXR0AhNnlAA.gif" width="600">
    <br>
    <i>Concatenation during Forward Propagation</i>
</div>
<br>

<a id='b'></a>
## 2. DenseNet Architecture

### 2.1. Basic DenseNet Composition Layer

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*IwvJGTxBAcb1H5tSJR6Lng.gif" width="600">
    <br>
    <i>Composition Layer</i>
</div>
<br>

각 layer는 __Pre-Activation Batch Norm(BN), ReLU, 3×3 Conv__를 통과해서 k개의 feature map이 결과값으로 나오도록 구성한다. 위 그림을 보면 x0,x1,x2,x3를 x4로 변환하는 과정이다. 이러한 아이디어는 _Pre-Activation ResNet_에서 가져왔다.

### 2.2. DenseNet-B (Bottleneck Layers)

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*dniz8zK2ClBY96ol7YGnJw.png" width="600">
    <br>
    <i>DenseNet-B</i>
</div>
<br>

모델의 복잡함과 크기를 줄이기위해, _BN-ReLU-3x3 Conv_이전 단계에서 _BN-ReLU-1x1 Conv_를 적용한 구조를 __DenseNet-B__라고 한다.

### 2.3. Multiple Dense Blocks with Transition Layers

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*BJM5Ht9D5HcP5CFpu8bn7g.png" width="600">
    <br>
    <i>Multiple Dense Blocks</i>
</div>
<br>

위의 DenseNet 구조를 묶어 Block 형태로 만들고, 여러개의 Block으로 만든 구조이다. Block 사이에는 1×1 Conv와  2×2 average pooling을 변형(transition) layer로 사용했다.

Feature map의 크기는 모든 dense block에서 동일하게 출력되므로 쉽게 연결(concatenate)할 수 있다.

Dense block 마지막에는 global average pooling와 softmax classifier를 추가한다.

### 2.4. DenseNet-BC (Further Compression)

논문에서는 모델을 좀더 간결하게(compat)만들기 위해 transition layer를 통과하면서 feature map의 숫자를 줄여보는 실험을 진행하였다.

Dense block이 m개의 feature maps를 가지고 있다면, transition layer에서는 $θm$개의 feature map을 생성한다고 하자. 여기서 $0<θ≤1$를 이용해서 압축 여부(compression factor)를 결정할 수 있다. $θ=1$ 이라면, transition layer를 통과해도 feature map의 수는 변하지 않는다. 

여기서 $ θ<1$의 값을 사용한 모델을 __DenseNet-C__하고 하며, 논문에서는 $θ=0.5$ 값을 사용했다. 위의 Bottleneck 구조와 $ θ<1$를 동시에 적용한 모델을 __DenseNet-BC__라고 한다.

실험 상에서는 B,C 구조의 적용 여부, layers 수 L, growth rate k를 변화시키면서 학습을 진행한다.

<a id='c'></a>
## 3. Advantages of DenseNet

### 3.1. Strong Gradient Flow

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*9atnQFu8ncrqFqZdB_LNVg.png" width="600">
    <br>
    <i>Implicit “Deep Supervision”</i>
</div>
<br>

Error 신호가 직접 각 layer로 전파되기 쉽워 Gradient Flow가 좋아지는 장점이 있다. 즉 각각의 layer도 마지막 분류 layer의 영향을 쉽게 받을 수 있으므로 'implicit deep supervision'라고 할 수 있다.

### 3.2. Parameter & Computational Efficiency

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*03pZkWqHN7A3pd81Pi-cIQ.png" width="600">
    <br>
    <i>
Number of Parameters for ResNet and DenseNet</i>
</div>
<br>

_ResNet_의 parameter의 수는 $C×C$에 비례하고, _DenseNet_에서는 $l×k×k$에 비례한다. 여기서 $k<<C$ 이므로 DenseNet이 ResNet보다 훨씬 작은 수의 parameter를 가질 수 있다.

### 3.3. More Diversified Features

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*VoaoQpASmgyaxEISacP44Q.png" width="600">
    <br>
    <i>More Diversified Features in DenseNet</i>
</div>
<br>

선행 layer의 모든 결과값을 입력으로 받기때문에 더 다양한 feature를 갖고 풍부한 패턴을 보이는 경향이 있다.

### 3.4. Maintains Low Complexity Features

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*PZHH8YgkG4nYlIpicCo1Ww.png" width="600">
    <br>
    <i>Standard ConvNet</i>
</div>
<br>

기존의 보통 ConvNet에서는 분류기가 가장 복잡한 features만 사용한다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*t_orlp67H-odvgMa4LTzzw.png" width="600">
    <br>
    <i>DenseNet</i>
</div>
<br>

_DenseNet_은 모든 단계(level)의 feature를 사용해서 분류한다. 이렇게하면  decision boundaries를 좀 더 부드럽게 만들며, 학습 데이터가 부족한 경우에도 좋은 성능을 보여줄 수 있다.

<a id='d'></a>
## 4. CIFAR & SVHN Small-Scale Dataset Results

### 4.1. CIFAR-10

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*DVC7dxFxs7ozyG6vJTOTsQ.png" width="600">
    <br>
    <i>CIFAR-10 Results</i>
</div>
<br>

_Pre-Activation ResNet_ 구조와 비교한 그림이다.

* Accuracy : Test Error를 살펴보면, DenseNet이 기존의 SOTA 모델의 결과보다 더 좋은 성능을 보여줬다.
* Capacity : compression과 bottleneck layer를 사용하지 않은 상태에서 모델의 Parameters가 증가할 수록 좋은 성능을 보여줬다. 이러한 결과를 통해 DenseNet구조에서는 크고 깊은 모델을 만들수록 모델의 성능이 증가하며, overfitting 문제나 optimization difficulties of residual networks 문제가 줄어든다고 볼 수 있다.
* Parameter Efficiency: 기존의 모델보다 Paremeter 수가 적으면서도 좋은 성능을 보여주는 것으로 보아, Parameter가 훨씬 효율적으로 사용되고 있다고 볼 수 있다.
* Overfitting : Paremeter 수가 작으면, overfitting 문제를 피하기 어려운 단점이 있다. 하지만 transition과 bottleneck 구조를 추가하게 되면 이러한 문제점을 해결하는데 도움을 준다.

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1250/1*f-wGcH50Bdx864n_biyjqg.png" width="800">
    <br>
    <i>C10+: Different DenseNet Variants (Left), DenseNet vs ResNet (Middle), Training and Testing Curves of DenseNet and ResNet (Right)</i>
</div>
<br>

* __Left__ : DenseNet-BC이 가장 좋은 성능을 보여줬다.
* __Middle__ : 동일한 error rate에서 DenseNet이 _Pre-Activation ResNet_보다 parameter 수가 3배 적다.
* __RIght__ : 0.8M parameters DenseNet과 10.2M parameters ResNet이 비슷한 성능을 보여주고 있다.

### 4.2. CIFAR-100

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*HkFC5vvy9Q9-kpMaZRcO5w.png" width="600">
    <br>
    <i></i>
</div>
<br>

### 4.3. Detailed Results

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*DyhtAHXX-kvuMtf421aSZg.png" width="600">
    <br>
    <i>
Detailed Results, + means data augmentation</i>
</div>
<br>

_SVHN_은 'Street View House Numbers' 데이터셋이다. 파란색은 가장 좋은 결과를 표시하고 있는데, SVHN 데이터셋에서는 basic DenseNet이 DenseNet-BC보다 좋은 성능을 보여주는 것을 볼 수 있다. 논문에서는 이 SVHM 데이터가 상대적으로 쉬운 데이터라서 더 깊은 모델인 DenseNet-BC에서 overfitting이 발생했다고 말한다.

<a id='e'></a>
## 5. ImageNet Large-Scale Dataset Results

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*HkVjuZm9vGh6GZmUi_o7AQ.png" width="300">
    <br>
    <i>
Different DenseNet Top-1 and Top-5 Error Rates with Single-Crop (10-Crop) Results
</i>
</div>
<br>

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*k75kOqrISzAEgtS15WWSZA.png" width="600">
    <br>
    <i>ImageNet Validation Set Results Compared with Original ResNet</i>
</div>
<br>

* __Left__ : 동일한 error rate에서 DenseNet이 _Pre-Activation ResNet_보다 parameter 수가 더 적은 것을 볼 수 있다.
* __RIght__ : 연산량의 관점에서도 비슷한 경향을 보이고 있다.

<a id='f'></a>
## 6. Further Analysis on Feature Reuse

<br>
<div align="center">
    <img src="https://cdn-images-1.medium.com/max/1000/1*dY47b5vX8S0MQcJbvIcFnQ.png" width="600">
    <br>
    <i>Heat map on the average absolute weights of how Target layer (l) reuses the source layer (s)</i>
</div>
<br>

그림에서 빨간 부분(red dot)이 l layer에서 s layer의 feature을 강하게 사용하고 있다는 것을 의미한다.

* 앞단의 layer에서 추출된 feature도 dense block을 통해 전달되므로 아래의(deeper) layer에서도 사용되고 있는 것을 볼 수 있다.
* transition layers도 weights를 분산시켜주는 모습을 볼 수있다(transition layer가 흐름을 방해하지 않는 것을 의미).
* 마지막 분류 layer에서도 전체적인 weights를 사용하고 있는 것을 확인할 수 있다.

<br>
<div align="center">
    <img src="" width="600">
    <br>
    <i></i>
</div>
<br>

## References

* [2017 CVPR] [DenseNet]<br>
Densely Connected Convolutional Networks